In [1]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as scila

from func import *
from math import pi

### Monomial Polynomials

Monomials is just a power series, where the $i^{th}$ coefficient is paired with the basis term of $x^i$. We can represent this as the Vandermonde matrix.

### Chebyshev Polynomials

The Chebyshev Polynomials are a basis of orthogonal polynomials. They are defined with 2 base cases and a recurrence of:

$$T_0(x) = 1, T_1(x) = x, T_{k+2}(x) = 2xT_{k-1}(x) - T_{k-2}(x)$$

### The Expansion Problem (Chebyshev to Monomial)

The Expansion problem, as defined by Bostan and Salvy, is converting a discrete set of points in $K$ from the Chebyshev basis to the Monomial basis. This can be done naively in $O(n^2)$ time by defining the matrix $F_{i,j}$ where the coefficient of $x^i$ is from $T_j$ as defined by the Chebshev basis.

In [2]:
def expansionFMat(n,a=2,b=1):
    assert(n > 0)
    F = np.zeros((n,n))
    F[0,0] = 1
    if(n == 1):
        return F
    
    F[1,1] = 1
    prev = np.asarray([0,1])
    curr = np.asarray([-1,0,2])
    nxt = np.zeros(3)
    for j in range(2,n):
        F[:j+1,j] = curr
        
        nxt = np.zeros(j+2)
        nxt[-len(curr):] = a*curr
        nxt[:len(prev)] -= b*prev
        prev = curr
        curr = nxt
        
    return F

In [7]:
n = 100

def rowExtremas(M):
    for i in range(len(M)):
        print("row {} max is {}, min is {}".format(i, max(M[i]), min((M[i]))))
    print("")

In [63]:
F = expansionFMat(n)
# rowExtremas(F)

# print(F)

x = np.random.random(n)

y = scla.solve_triangular(F,x,lower=False)
print(la.norm(np.dot(F,y) - x))
print("Chebyshev Transform:",la.norm(F,ord=2))

0.008574616713695304
Chebyshev Transform: 1.2283680538636764e+37


### New Recurrance

For numerical stability reasons, we define the new recurrance to be:

$$T_0(x) = 1, T_1(x) = x, T_{k+2}(x) = xT_{k-1}(x) - \frac{1}{100}T_{k-2}(x)$$

In [20]:
Fsmall = expansionFMat(n, 1, 0.01)

# rowExtremas(Fsmall)

x = np.random.random(n)

y = scla.solve_triangular(Fsmall,x,lower=False)
print(la.norm(np.dot(Fsmall,y) - x))
print("Chebyshev Transform:",la.cond(Fsmall,p=2))

4.165186303040984e-15
Chebyshev Transform: 15.64675568439878


### Evaluating

With the new set of orthogonal polynomials, we need to evaluate terms at a specific time, i.e.

$$f(t_j) = \sum\limits_{i}^{d-1} c_i(t_j)^i$$

In our new basis represented by the upper-triangular matrix $F$, the different rows represent the coefficients for $t^j$ in ascending order down (so the bottom has the largest coefficient), and the columns represent the different polynomials $P_i$ in increasing fashion.

For instance, to get the first $4$ Chebyshev Polynomials, we get the matrix

$$
\begin{bmatrix}
 1 &  0 & -1 &  0 \\
 0 &  1 &  0 & -3 \\
 0 &  0 &  2 &  0 \\
 0 &  0 &  0 &  4
\end{bmatrix}
$$

The first four Chebyshev polynomials are

$$1, \ t, \ 2t^2 - 1, \ 4t^3-3t$$

In [21]:
def evaluateBasis(B,t,i):
    """
    @B: matrix basis
    @t: time to evaluate at
    @i: the i-th polynomial
    """
    n = len(B)
    total = 0
    for e in range(n):
        total += B[e,i] * t**e
    return total

### Testing Against Chebyshev Basis

In [45]:
n = 100
chebyStd = chebyMatrix(n)

In [46]:
def evalPolynomialVander(B,full=False):
    n = len(B)
    z = n if full else 2*n-1
    nodes = chebyNodes(z)
    # nodes = np.linspace(1,-1,z)
    V = np.zeros((z,n))
    for i in range(z):
        for j in range(n):
            V[i,j] = evaluateBasis(B,nodes[i],j)
    return V

In [47]:
chebyExp = expansionFMat(n)
chebyV = evalPolynomialVander(chebyExp)
      
print("Expansion Conditioning:", la.norm(chebyExp, ord=2))
print("Interoplation Conditioning:", la.norm(chebyV, ord=2))
print("Error:",la.norm(chebyV - chebyStd, ord='f')/la.norm(chebyStd, ord='f'))

Expansion Conditioning: 1.2283680538636764e+37
Interoplation Conditioning: 5.357964560677981e+21
Error: 5.815528726189489e+19


### Testing Small Orthgonality

In [48]:
smallBasis = expansionFMat(n,1,0.01)
chebySmallV = evalPolynomialVander(smallBasis)

print("Expansion Conditioning:", la.norm(smallBasis, ord=2))
print("Interpolation Conditioning:", la.norm(chebySmallV, ord=2))

Expansion Conditioning: 6.994743601691826
Interpolation Conditioning: 22.91415864924521


In [49]:
print(smallBasis)

[[ 1.0000e+00  0.0000e+00 -1.0000e+00 ...  0.0000e+00 -1.0000e-96
   0.0000e+00]
 [ 0.0000e+00  1.0000e+00  0.0000e+00 ...  4.8010e-93  0.0000e+00
  -4.9010e-95]
 [ 0.0000e+00  0.0000e+00  2.0000e+00 ...  0.0000e+00  1.1765e-91
   0.0000e+00]
 ...
 [ 0.0000e+00  0.0000e+00  0.0000e+00 ...  2.0000e+00  0.0000e+00
  -2.9300e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 ...  0.0000e+00  2.0000e+00
   0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 ...  0.0000e+00  0.0000e+00
   2.0000e+00]]


### Toom-Cook

We use our small orthogonal polynomial to do Toom-Cook with one level of recursion

In [50]:
# just single level
def orthoInterpolate(f,g):
    n = len(f)
    assert(n == len(g))
    
    # compose to orthogonal basis
    composeBasis = expansionFMat(n,1,0.01)
    f2 = scla.solve_triangular(composeBasis,f,lower=False)
    g2 = scla.solve_triangular(composeBasis,g,lower=False)

    # evaluate 
    V = evalPolynomialVander(composeBasis)
    p = np.dot(V,f2)
    q = np.dot(V,g2)

    # multiply
    r = p*q

    # interpolate
    decomposeBasis = expansionFMat(2*n-1,1,0.01)
    Vinter = evalPolynomialVander(decomposeBasis,full=True)
    I = la.solve(Vinter, r)

    # decompose back to monomial
    return np.dot(decomposeBasis,I)

In [51]:
f = np.random.random(n)
g = np.random.random(n)

conv = orthoInterpolate(f,g)
convLib = np.convolve(f,g)

print(la.norm(conv-convLib)/la.norm(convLib))

12687.98676348165


In [52]:
def errorAnalysis(result,control):
    assert(len(result) == len(control))
    for i in range(1,len(control)+1):
        print("Total Error for {} elements".format(i),":",la.norm(result[:i]-control[:i])/la.norm(control[:i]))
    print("")

    for i in range(len(control)):
        print("Element Error for idx {}".format(i),":",la.norm(result[i]-control[i])/la.norm(control[i]))
    print("")

In [53]:
# errorAnalysis(conv,convLib)

composeBasis = expansionFMat(n,1,0.01)
V = evalPolynomialVander(composeBasis)
decomposeBasis = expansionFMat(2*n-1,1,0.01)
Vinter = evalPolynomialVander(decomposeBasis,full=True)
    
print("Compose conditioning:",la.norm(composeBasis,ord=2))
print("Evaluate conditioning:",la.norm(V,ord=2))
print("Decompose conditioning:",la.norm(decomposeBasis,ord=2))
print("Interpolate conditioning:",la.norm(Vinter,ord=2))

Compose conditioning: 6.994743601691826
Evaluate conditioning: 22.91415864924521
Decompose conditioning: 18.07655574065172
Interpolate conditioning: 23.325273247217634


### Sources of Error

While the system is quite well-conditioned, we still see the classical errors of as [presented here](https://math.stackexchange.com/questions/200924/why-is-lagrange-interpolation-numerically-unstable).

This begs two questions:

1. Is our small orthogonal polynomials even orthogonal
2. Or is it inherently bad to convolve?

In [41]:
composeBasis = expansionFMat(n,1,0.01)
f2 = scla.solve_triangular(composeBasis,f,lower=False)
frecover = np.dot(composeBasis,f2)

print(la.norm(f - frecover)/la.norm(f))

1.410289342156766e-16


### Comparison to Chebyshev Polynomial Conversion

We compare the use of small orthogonal polynomials to our change of basis to just Chebyshev Polynomials

In [72]:
exF = expansionFMat(n)
exF2 = expansionFMat(2*n-1)

z = 2*n-1
i = np.arange(n, dtype=np.float64)
j = np.arange(z, dtype=np.float64)
# Chebyshev nodes:
nodes = np.cos((2*j+1)/(2*z)*np.pi)
V = np.vander(nodes,N=n,increasing=True)
V2 = np.vander(nodes,increasing=True)
sb = np.dot(V,f) * np.dot(V,g)
V3 = np.cos(j * np.arccos(nodes.reshape(-1, 1)))
s = la.solve(V3,sb)
c = np.dot(exF2,s)

# convert to Chebshev
fc = scla.solve_triangular(exF,f,lower=False)
gc = scla.solve_triangular(exF,g,lower=False)

# chebshev interpolation
vc = chebyInterpolate(fc,gc)

# convert back
conv = np.dot(exF2,vc)

# control
convLib = np.convolve(f,g)

print("Error:",la.norm(conv-convLib)/la.norm(convLib))
print("")
# errorAnalysis(conv,convLib)
print("Error2:",la.norm(convLib-c)/la.norm(convLib))
print(c)
print(convLib)

Error: 6.745607244150876e-06

Error2: 6.324206194888517e-06
[0.57317641 0.675038   0.9332859  1.26916031 1.6550433  2.08237118
 2.35987839 2.16396176 2.58484007 2.93406777 2.57171889 3.99332635
 3.51478369 3.56830158 4.59755768 4.27465926 4.33404893 4.34749152
 3.39977172 3.25835676 3.15189961 2.04331765 2.71386128 2.60019285
 1.69522931 1.86828311 1.44836266 1.04824543 1.1920618  0.82349059
 0.12493627]
[0.57317641 0.675038   0.9332859  1.26916031 1.6550433  2.08237118
 2.35987839 2.16396177 2.58484009 2.9340677  2.57171866 3.99332671
 3.51478506 3.56830039 4.59755215 4.27466183 4.33406484 4.34748809
 3.39973865 3.25835893 3.15194943 2.04331856 2.71380745 2.60018975
 1.69526996 1.86828582 1.44834231 1.04824429 1.19206787 0.82349079
 0.12493545]


### Standard Toom-Cook

In [20]:
conv = TCconv2(f,g,k=n)

print("Error:",la.norm(conv-convLib)/la.norm(convLib))
print("")
# errorAnalysis(conv,convLib)

Error: 4.979095072610425e-11



So the methodology is not better than Monomial interpolation.

### Possible Sources of Error

While the Chebyshev Interpolation error should be well-conditioned, the stability of converting brings some unfriendly errors. Particularly, converting from one basis to another is ill-conditioned.

When compared to the Vandermonde matrix of the monomial interpolation, the Chebyshev coefficents are quiet bad.

In [19]:
print("Vandermonde Conditioning:",la.norm(np.vander(rs(n),increasing=True),ord=2))

print("Decomposition Conditioning:",la.norm(exF,ord=2))

print("Expansion Conditioning:",la.norm(exF2,ord=2))

Vandermonde Conditioning: 23423.763539533997
Decomposition Conditioning: 142.378502437558
Expansion Conditioning: 57813.677997307335
